In [1]:
from build import CppModule
import numpy as np
import timeit



In [2]:

def convolution(img_matrix, filter, conv_stride = 1):
    conv = np.zeros((1, (img_matrix.shape[1] - filter.shape[1])//conv_stride + 1))
    for i in range(0,img_matrix.shape[0] - filter.shape[0] + 1, conv_stride):
        row = np.array([])
        for k in range(0,img_matrix.shape[1] - filter.shape[1] + 1, conv_stride):
            res = np.sum(img_matrix[i : i + filter.shape[0], k : k + filter.shape[1]] * filter)
            row = np.append(row, res).reshape(1, -1)
        conv = np.vstack((conv, row))
    conv = conv[1:,:]
    return conv


In [3]:
def Z_convolution(X, K):
    kernel_size = K.shape[-1]
    output = np.zeros((X.shape[0], K.shape[0],X.shape[2] - kernel_size + 1, X.shape[3] - kernel_size + 1))
    for i in range (X.shape[0]):
        for j in range (K.shape[0]):
            sum = 0
            for k in range (X.shape[1]):    
                sum += convolution(X[i][k],K[j][k])
            output[i][j] = sum 

    return output

In [4]:
def conv_3d(input, kernel, kernel_size, n_kernels):
    output = np.zeros((n_kernels, input.shape[0], input.shape[1] - kernel_size + 1, input.shape[2] - kernel_size + 1))
    for i in range (n_kernels):
        for k in range(input.shape[0]):
            output[i][k]= convolution(input[k], kernel[i])
    return output

In [5]:
def convolution(img_matrix, filter, conv_stride = 1):
    conv = np.zeros((1, (img_matrix.shape[1] - filter.shape[1])//conv_stride + 1))
    for i in range(0,img_matrix.shape[0] - filter.shape[0] + 1, conv_stride):
        row = np.array([])
        for k in range(0,img_matrix.shape[1] - filter.shape[1] + 1, conv_stride):
            res = np.sum(img_matrix[i : i + filter.shape[0], k : k + filter.shape[1]] * filter)
            row = np.append(row, res).reshape(1, -1)
        conv = np.vstack((conv, row))
    conv = conv[1:,:]
    return conv
def padded(matrix, n_pads = 1):
    res = np.zeros((matrix.shape[0] + n_pads * 2, matrix.shape[1] + n_pads * 2))
    res[n_pads : res.shape[0] - n_pads, n_pads : res.shape[1] - n_pads] = matrix
    return res
def dX_convolution(dZ, K):
    kernel_size = K.shape[-1]
    dX = np.zeros((dZ.shape[0], K.shape[1],dZ.shape[2] + kernel_size - 1, dZ.shape[3] + kernel_size - 1))
    for i in range (dZ.shape[0]):
        for j in range (K.shape[0]):
            for k in range (K.shape[1]):
                dX[i][k] = convolution(padded(dZ[i][j], n_pads=kernel_size - 1),np.rot90(K[j][k], 2))
    return dX

In [6]:
def max_pooling(input, pool_size = 2):
    pool_stride = pool_size
    conv2 = np.zeros((1, (input.shape[1] - pool_size)//pool_stride + 1))
    for i in range(0,input.shape[0] - pool_size + 1, pool_stride):
        row = np.array([])
        for k in range(0,input.shape[1] - pool_size + 1, pool_stride):
            res = np.max(input[i : i + pool_size,k : k + pool_size])
            row = np.append(row, res).reshape(1, -1)
        conv2 = np.vstack((conv2, row))
    conv2 = conv2[1:,:]
    return conv2

In [7]:
dZ = np.ones((10,5, 28,28))
K = np.ones((5,3,3,3))
input = np.array([[ 5,  1,  20,  30],
                     [ 4,  20,  40,  7],
                     [ 80,  9, 10, 11],
                     [12, 8, 140, 15]])
dP = np.array([[1,2], [3,4]])

In [11]:

start = timeit.default_timer()
# All the program statements
P = CppModule.avg_pooling(input, 2)
print(P)
print(CppModule.avgpool_de(input, dP, 2))
stop = timeit.default_timer()
execution_time = stop - start

print("C++ code executed in "+str(execution_time))

[[ 7.5  24.25]
 [27.25 44.  ]]
[[0.25 0.25 0.5  0.5 ]
 [0.25 0.25 0.5  0.5 ]
 [0.75 0.75 1.   1.  ]
 [0.75 0.75 1.   1.  ]]
C++ code executed in 0.0028791959994123317


In [9]:
# CppModule.Z_convolution(X, K)

In [10]:
start = timeit.default_timer()

# All the program statements
print(max_pooling(input, 4))
stop = timeit.default_timer()
execution_time = stop - start

print("Python code executed in "+str(execution_time))

[[140.]]
Python code executed in 0.003856982999423053
